In [1]:
import csv
import os
import polars as pl
import Enum_data as ed
import general_functions as gf
import operational_functions as of
import monthly_report_functions as mf
import income_statement_functions as isf
import others_functions as otf
import overhead_cost_functions as ovf
import quantity_turnover_functions as qf
import raw_material_functions as rf
import cogs_functions as cf

from persiantools import characters, digits

In [2]:
def run_program(symbol: str, year: int = 1400):
    report_types = [value.value for value in ed.names]
    all_reports = {}

    financial_years=gf.get_financial_years(symbol, year)
    for year in financial_years:
        parse_date = gf.parse_date_persian(year)
        dummy_dict = {}
        for report in report_types:
            report_type = ed.names[report].value
            sheet_num = ed.sheets[report_type].value
            #table = ed.tabels[report_type].value
            reports = gf.get_results(symbol, parse_date, report_type, sheet_num)
            dummy_dict[report_type] = reports
        all_reports[year] = dummy_dict
    return all_reports

In [5]:
symbol = 'شپدیس'
all_reports = run_program(symbol)

In [6]:
filtered_all_reports = {}
for fin_year in sorted(list(all_reports.keys()), reverse=True):
    filtered_all_reports[fin_year] = {}
    for report_type in [value.value for value in ed.names]:
        if report_type in ['Monthly_report']: continue
        if fin_year in all_reports.keys():
            if len(all_reports[fin_year][report_type]) > 0:
                if all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]['period'] == 12:
                    filtered_all_reports[fin_year][report_type] = all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]
                else:
                    filtered_all_reports[fin_year][report_type] = all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]


In [7]:
final_filtered_all_reports = {}
for key, val in filtered_all_reports.items():
    if len(val) > 0:
        date = list(all_reports[key]['Operational'].keys())[-1]
        if int(key) == int(date) : 
            final_filtered_all_reports[key] = val
        else:
            final_filtered_all_reports[str(date)] = val

In [ ]:
all_data = {}
for year, report_types in final_filtered_all_reports.items():
    for report_type, st_dic in report_types.items():
        if report_type == "Monthly_report":
            continue

        date = gf.extract_date(st_dic["title"])
        period = st_dic["period"]
        publish = st_dic["publish"]
        url = st_dic["url"]

        if report_type == 'Operational':
            continue
            d = of.create_operational_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data, report_type, d)
        elif report_type == 'RawMaterial' :
            continue
            #Fix
            d = rf.creat_raw_material_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data, report_type, d)
            
        elif report_type == 'COGS' :
            continue
            d = cf.create_cogs_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data,report_type, d)
        elif report_type == 'Cost' :
            continue
            #Fix
            d = ovf.creat_Overhead_Cost_dataframe(symbol, url, date, period, publish, report_type)
            all_data = gf.sort_df_dic(all_data, report_type, d)
        elif report_type == 'Incoeme_Statment' :
            continue
            d = isf.create_Incoeme_Statment_dataframe(symbol, url, date, period, publish)
            all_data =gf.sort_df_dic(all_data , report_type , d)
        elif report_type == 'Others' :
            continue
            d = otf.create_Others_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data, report_type , d)
        elif report_type == 'Overhead' :
            #Fix
            d = ovf.creat_Overhead_Cost_dataframe(symbol, url, date, period, publish, report_type)
            all_data = gf.sort_df_dic(all_data, report_type, d)
        

In [8]:
all_data['Overhead'].keys()

dict_keys([14030631, 14040631, 14020631, 14010631, 14000631])

#### Excel

In [ ]:
df.to_pandas().to_excel(
    "output.xlsx",
    sheet_name="Sheet1",
    index=False
)

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

out = "output.xlsx"

# --- 1) Create a real workbook with a visible sheet (prevents the error) ---
wb = Workbook()
ws = wb.active
ws.title = "Sheet1"
ws.sheet_state = "visible"
ws.sheet_view.rightToLeft = True
wb.save(out)

titles = ["تولید", "فروش", "نرخ"]
tables = [t1, t2, t3]
gap = 2
excel_row = 1  # 1-based Excel row

with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    ws = writer.book["Sheet1"]

    for title, table in zip(titles, tables):
        date_row = table.select("Date").to_pandas().iloc[0].tolist()
        pdf = table.drop("Date").to_pandas()

        ncols = pdf.shape[1]
        last_col = get_column_letter(ncols)

        # title (merged)
        ws.merge_cells(f"A{excel_row}:{last_col}{excel_row}")
        ws[f"A{excel_row}"].value = title
        ws[f"A{excel_row}"].font = Font(bold=True)
        ws[f"A{excel_row}"].alignment = Alignment(horizontal="center")

        # date row (between title and header)
        for i, v in enumerate(date_row, start=1):
            ws.cell(row=excel_row + 1, column=i, value=v)

        # table (header + data) under date row
        pdf.to_excel(writer, sheet_name="Sheet1", index=False, startrow=excel_row + 1)

        # advance: title + date + header + data + gap
        excel_row += 3 + len(pdf) + gap
